In [12]:
from bs4 import BeautifulSoup, SoupStrainer
import httplib2
import re
import pandas as pd
from itertools import chain
import time 
from google.colab import files

In [13]:
# setting up dataframe with columns required
joke_df = pd.DataFrame(columns=['code', 'title', 'text', 'image captions', 'rating', 'state', 'tags',  'link'], index=None)

In [14]:
http = httplib2.Http()

In [ ]:
# scraping joke scp pages based on directory of all joke scps

# get all links first then loop through, maybe preserve as tag to then get code from there more easily etc.
from itertools import chain

status, directory_response = http.request('https://scp-wiki.wikidot.com/joke-scps')

directory_soup = BeautifulSoup(directory_response, 'html.parser')

links = [ i.find_all('a', href=True) for i in directory_soup.find_all('div', {'class': 'content-panel'})]
# remove theme links/not actual links
links = list(chain.from_iterable(links))

links = [i for i in links if 'theme:' not in i['href'] and i.text[-1] != ':' and i.text not in ['Home Page', 'Bearitage Collection']]
print(links)

In [ ]:
print(len(links))

In [ ]:
for c, link in enumerate(links):
  # timeout to limit load on the site (based on api requests per minute limit)
  if c % 240 == 239:
    print(c)
    time.sleep(60)

  # requesting page this tag refers to
  url = 'https://scp-wiki.wikidot.com/' + link['href']
  status, response = http.request(url)
  soup = BeautifulSoup(response, 'html.parser')

  # getting title from directory page
  if ' - ' in link.parent.text:
    title = link.parent.text.split(' - ')[-1]
    code = link.parent.text.split(' - ')[0]
  else:
    title = link.text 
    # could also remove 'codename:' part from some of them but probably something for later in data processing conditional on how want to use
    code = None
  
  captions = []
  state = None
  rating = None

  # if soup.find(lambda x: x.name=="a" and "clearance credentials" in x.text): 
  # checking if there's any restricted by clearance - there aren't so no need to handle
  #   print('RESTRICTED!!!!')

  if soup.find('div', {'id': 'u-adult-warning'}) != None:
    # print("ADULT!!!!") # initial check to see if this set contains any adult restricted articles - which it does so need to handle
    url = 'https://scp-wiki.wikidot.com/adult:{}/noredirect/true'.format(link['href'][1:])

    status, adult_response = http.request(url)
    soup_adult = BeautifulSoup(adult_response)

    state = "age restricted"

    rating_tag = soup_adult.find(class_ = 'number prw54353')

    if rating_tag != None:
      rating = rating_tag.text  

    text = [i for i in soup_adult.find(id = 'page-content').find_all('p')]

    tags_raw = soup_adult.find('div', {'class': 'page-tags'}).find('span').find_all('a')
    tags = " ".join([i.text for i in tags_raw]).replace(",", "")
  else:
    # getting text for the scp within the main content div
    text = [i for i in soup.find(id = 'page-content').find_all('p')]
    
  # p tags include image captions which are excluded as below, may be other similar cases left in
  for i in text:
    try:
      if i.parent.attrs['class'][0] == 'scp-image-caption':
        captions.append(i.text)
        text.remove(i)
    except:
      pass
      
  # getting text from result set
  text = [i.text for i in text]

  # getting the rating
  if rating == None: # accounting for ones that dont have ratings on main page, mostly due to adult content notice page first
    rating = soup.find(class_ = 'number prw54353')
    if rating == None:
      pass
    else:
      rating = rating.text

  # getting tags
  if not state:
    tags_raw = soup.find('div', {'class': 'page-tags'}).find('span').find_all('a') if soup.find('div', {'class': 'page-tags'}) != None else None
    tags = " ".join([i.text for i in tags_raw]).replace(",", "") if tags_raw != None else None
  else:
    # just in case useful tags spread across the two pages here getting from restricted notif page too and adding
    tags_raw = soup.find('div', {'class': 'page-tags'}).find('span').find_all('a')
    tags = " ".join(list(set([i.text for i in tags_raw] + tags.split()))).replace(",", "")
  
  # quotes to preserve commas and not cause csv issues - joining up p tags with newlines
  text = "\"{}\"".format(" \n ".join([i.strip() for i in text if len(i.strip()) > 0])) if len(text) > 0 else None 

  captions = "\"{}\"".format(" \n ".join(i for i in captions)) if len(captions) > 0 else None

  # setting status - age restricted will already have been assigned
  if state:
    pass
  elif text == None:
    state = "missing"
  elif text and "This page may have been moved or deleted" in text:
    state = "deleted"
  elif "[blocked]" in title.lower():
    state = "blocked"
  else:
    state = "active" # this will catch any anomalies too

  info = [code, "\"{}\"".format(title), text, captions, rating, state, tags, url]
  joke_df.loc[len(joke_df)] = info

joke_df.to_csv('scpjoke.csv', index=False) 

# auto download after saving to file - with timeout to try and ensure the file will be available/ complete
time.sleep(100)
files.download('scpjoke.csv') 